In [1]:
import json
import spacy
from spacy import displacy
import random
from nltk.corpus import stopwords

In [2]:
stop_words_english = set(stopwords.words('english'))


In [3]:
with open("/home/leon/Desktop/market/data/all_data2.json", "r") as f:
    data = json.load(f)

In [4]:
data["reviews"][1]["review_text"]

'obsessed!!!'

In [5]:
def extract_words(text, spacy_object, stop_words):
    keywords = []
    doc_object = spacy_object(text)
    for token in doc_object:
        if token.pos_ == "ADJ":
            if token.dep_ in ["amod", "conj"]:
                keywords += [token]

        if token.dep_ == "nsubj":
            for child in token.head.children:
                if child.pos_ == "ADJ":
                    keywords += [child]

        if token.dep_ in ["dobj", "acomp"]:
            keywords += [token]
            
        if token.dep_ == "conj" and token.head.pos_ == "VERB":
            keywords += [token]

        if token.dep_ == "advmod" and token.pos_ == "ADV" and token.head.pos_ == "ADJ":
            keywords += [token]
        
        if len([c for c in token.children]) > 3 and token.text not in stop_words:
            keywords += [token]
            
    if len(keywords) == 0:
        keywords = list(doc_object)

    if len(doc_object) == 0 :
        keywords = list(spacy_object("and"))
        
    return(list(set(keywords)))

In [6]:
nlp = spacy.load('en_core_web_md')

In [7]:
index = random.randint(1,5000)
ex = data["reviews"][index]["review_text"]
doc = nlp(ex)
ex

'My pores were dramatically smaller. It’s a little itchy when it’s on from the carbonated sensation. Makes you giggle, fun to try with your partner or a friend. Quality product and great stocking stuffer. It helped my redness and breakouts too.'

In [8]:
extract_words(ex, nlp, stop_words_english)


[great,
 product,
 smaller,
 itchy,
 ’s,
 helped,
 ’s,
 redness,
 fun,
 Makes,
 dramatically,
 little,
 Quality]

In [9]:
displacy.render(doc, style='dep', jupyter=True)

In [10]:
import numpy as np
import pandas as pd

In [11]:
def get_vector(text, spacy_object, stop_words):
    
    words = extract_words(text, spacy_object, stop_words)
    table = np.array([word.vector for word in words])
    review_vector = np.mean(table, 0)
    
    return (review_vector)


In [12]:
vector_data = []
for x in np.arange(len(data["reviews"])):
    if x % 1000 == 0:
        print(x)
    
    try:
        review_vector = get_vector(data["reviews"][x]["review_text"], nlp, stop_words_english)
        vector_data += [review_vector]
    except:
        print(data["reviews"][x]["review_text"])
        break
    
    

0
1000
2000
3000
4000
5000


In [13]:
any([len(x) != 300 for x in vector_data])

False

In [14]:
dataframe = pd.DataFrame(np.array(vector_data),columns=np.arange(len(vector_data[0])))

In [15]:
dataframe.to_csv("/home/leon/Desktop/market/data/vector_data_mean.csv", header=True, index=True)

In [16]:
dataframe = pd.read_csv("/home/leon/Desktop/market/data/vector_data_mean.csv", header=0, index_col=0)

In [17]:
ratings = [float(data["reviews"][x]["review_rating"])*30 for x in np.arange(len(data["reviews"]))]

In [18]:
ratings = pd.DataFrame(ratings)

In [19]:
master_table = pd.concat([ratings, dataframe], axis=1)

In [20]:
master_table.columns = [x for x in np.arange(301)]
master_table.to_csv("/home/leon/Desktop/market/data/vector_data_ratings_mean.csv", header=True, index=True)